In [3]:
import requests
from lxml import etree
import time
import random
import ssl
import json
import pandas as pd

In [4]:
# use spider to get the content of the website
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}

html = requests.get(
            url=url,
            headers=headers
        ).content.decode('utf-8','ignore')

dd = etree.HTML(html)



In [51]:
# get Postal_code by Xpath
# the responce contains '\n', I ues 'replace' to get rid of them. 
Postal_code_html = dd.xpath('/html/body/div[3]/div[3]/div[5]/div[1]/table[1]/tbody/tr/td[1]/text()')
Postal_code = []
for i in Postal_code:
    i = i.replace('\n','')
    Postal_code.append(i)


In [52]:
Borough_html = dd.xpath('/html/body/div[3]/div[3]/div[5]/div[1]/table[1]/tbody/tr/td[2]/text()')
Borough = []
for i in Borough_html:
    i = i.replace('\n','')
    Borough.append(i)


In [53]:
Neighbourhood_html = dd.xpath('/html/body/div[3]/div[3]/div[5]/div[1]/table[1]/tbody/tr/td[3]/text()')
Neighbourhood = []
for i in Neighbourhood_html:
    i = i.replace('\n','')
    Neighbourhood.append(i)


In [62]:
Toronto = {'Postal Code':Postal,'Borough':Borough,'Neighbourhood':Neighbourhood}


In [92]:
# transform to dataframe
Toronto_df = pd.DataFrame(Toronto)


In [91]:
# drop 'Not assigned'
#pd.set_option('display.max_rows', None)

df = Toronto_df
df=df[~df['Borough'].isin(['Not assigned'])]

In [65]:
df.shape


(103, 3)

In [89]:
# Read the csv
latlong=pd.read_csv('Geospatial_Coordinates.csv')


In [90]:
# Merge dateframe
Tor = pd.merge(df,latlong,on='Postal Code')


### Now, let's explore Toronto

In [70]:
# the shape of the data
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Tor['Borough'].unique()),
        Tor.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


### Mapping the data

In [80]:
# get the latitude and longtitude of Toronto
import folium
from geopy.geocoders import Nominatim

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


In [88]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Tor['Latitude'], Tor['Longitude'], Tor['Borough'], Tor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto